In [1]:
import pandas as pd
import numpy as np
import random
import collections

In [2]:
def inputs(path):
    X = pd.read_csv(path,encoding='gb2312')
    y = X.pop("Label")
    return X,y

In [3]:
X,y = inputs("D:/data/train.csv")

In [4]:
def inputs_test(path):
    X_test = pd.read_csv(path,encoding='gb2312')
    Id = X_test.pop("SmsId")
    return X_test,Id

In [5]:
X_test, Id = inputs_test("D:/data/test.csv")

In [6]:
shape = X.shape[0]  # can use for the cache
test_shape = X_test.shape[0]
X = [X.iloc[i][0] for i in range(shape)]

X_test = [X_test.iloc[i][0] for i in range(test_shape)]

### 使用 Naive Bayes 进行文本分类

## 如何解决文本数据维度不一致，无法构造矩阵的问题？

## 因无法解决此问题，终于还是选择了调用 scikit-learn 的 API T_T

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

def get_Xtf(X):

    # 词出现的次数

    count_vect = CountVectorizer()
    X_counts = count_vect.fit_transform(X)

    # 词频

    tf_transformer = TfidfTransformer(use_idf=False).fit(X_counts)
    X_tf = tf_transformer.transform(X_counts)

    X_tf = tf_transformer.transform(X_counts)
    return X_tf


In [14]:
X_tf = get_Xtf(X+X_test)

D:\Python\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [15]:
X_tf.shape

(6687, 8628)

In [16]:
X_train_tf = X_tf[:shape,:]
X_test_tf = X_tf[shape:,:]

In [19]:
y_train = y

In [21]:
from sklearn.naive_bayes import MultinomialNB
y_train= np.reshape(y_train,(y_train.shape[0],1))
clf = MultinomialNB().fit(X_train_tf, y_train)

D:\Python\anaconda\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
from sklearn import metrics

pred = clf.predict(X_test_tf)

#print(metrics.classification_report(y_test,pred))

In [23]:
pred.shape

(1115,)

In [25]:
X_test_tf.shape

(1115, 8628)

In [26]:
pred[:30]

array(['ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'spam', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham'], dtype='<U4')

In [28]:
save = pd.DataFrame({"SmsId":Id,"Label":pred})

In [29]:
save.to_csv("D:/data/MySubmission.csv")